In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key (required if using OpenAI): "
    )

base_url = os.environ.get("BASE_URL")
model_name = os.environ.get("MODEL_NAME")
deepseek_model_name = os.environ.get("DEEPSEEK_MODEL_NAME")

print(base_url, model_name)

https://dashscope.aliyuncs.com/compatible-mode/v1 qwen-plus


In [2]:
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model=deepseek_model_name,
    temperature=0,
    max_retries=2,
    base_url=base_url,
)

In [3]:
response = llm.invoke("Hello")
response.content

'Hello! 😊 How can I assist you today?'

In [4]:
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """Get the weather from a location."""

    return "Sunny."


llm_with_tools = llm.bind_tools([get_weather])

response = llm_with_tools.invoke("What's the weather in San Francisco, CA?")
response.content

''

In [10]:
# 检查是否有工具调用
if response.tool_calls:
    # 执行工具调用
    tool_call = response.tool_calls[0]
    result = get_weather.invoke(tool_call["args"])
    print("Tool result:", result)

Tool result: Sunny.


In [11]:
from langchain_core.output_parsers import StrOutputParser

chain = llm | StrOutputParser()

In [12]:
response = chain.invoke("Hello")
response

'Hello! 😊 How can I assist you today?'

In [13]:
for chunk in chain.stream("What's the weather in San Francisco, CA?"):
    print(chunk, end="|")

|I| can|'t| fetch| real|-time| weather| data|,| but| you| can| check| the| current| weather| in| **|San| Francisco|,| CA|**| using| services| like|:|  

|-| [|Weather|.com|](|https|://|weather|.com|)|  
|-| [|Acc|u|Weather|](|https|://|www|.|acc|uwe|ather|.com|)|  
|-| [|National| Weather| Service|](|https|://|www|.|weather|.gov|)|  
|-| A| quick| search| on| Google| ("|weather| in| San| Francisco|")|  

|Would| you| like| general| climate| information| for| San| Francisco| instead|?| It| typically| has| mild| temperatures|,| frequent| fog| (|especially| in| summer|),| and| cool| bree|zes| from| the| Pacific|.| Let| me| know| how| I| can| help|!| 🌁|🌤|️||

In [8]:
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("anthropic:claude-3-5-sonnet-latest")


In [9]:
llm_with_tools = model.bind_tools([get_weather])


In [10]:
llm_with_tools.invoke("What's the weather in San Francisco, CA?")

AIMessage(content=[{'text': "I'll check the weather in San Francisco for you.", 'type': 'text'}, {'id': 'toolu_013Y8pTbRWpqrR4aWDmGQc8U', 'input': {'location': 'San Francisco, CA'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01VCmBaAdmdbtJyfZCT4qST9', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 384, 'output_tokens': 68, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--7493701a-f813-4a4f-8118-650cad05af0b-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco, CA'}, 'id': 'toolu_013Y8pTbRWpqrR4aWDmGQc8U', 'type': 'tool_call'}], usage_metadata={'input_tokens': 384, 'output_tokens': 68, 'total_tokens': 452, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

## 模型差异对比
DeepSeek 模型的行为：
```python
# DeepSeek 的响应
response.content = ""  # 空字符串
response.tool_calls = [{'name': 'get_weather', 'args': {...}}]
```
Claude 模型的行为：
```python
# Claude 的响应  
response.content = [
    {'text': "I'll check the weather in San Francisco for you.", 'type': 'text'},
    {'id': '...', 'input': {...}, 'name': 'get_weather', 'type': 'tool_use'}
]
response.tool_calls = [{'name': 'get_weather', 'args': {...}}]
```
###关键区别
- DeepSeek：当决定调用工具时，完全不返回文本内容，只生成工具调用
- Claude：既提供解释性文本（"I'll check the weather..."），又生成工具调用